#Semana 7 – Operaciones avanzadas en DataFrames: Filtrado, Agrupamiento y Estadísticas Agregadas

**Objetivo:**

Al finalizar esta clase, el/la estudiante será capaz de:

* Aplicar condiciones complejas para filtrar información crítica.

* Agrupar datos por múltiples claves jerárquicas.

* Implementar funciones agregadas simultáneas y personalizadas con agg().

* Integrar estos conocimientos en un sistema analítico orientado a objetos en contexto empresarial.



**Un sprint es un período de tiempo fijo (generalmente entre 1 y 4 semanas) durante el cual un equipo de desarrollo trabaja para completar un conjunto específico de tareas o funcionalidades que se han planificado previamente.**

# Simulación de Dataset Empresarial Complejo

**Contexto:**

Empresa de desarrollo de software con proyectos modulares, múltiples equipos y costos variables por sprint.


In [ ]:
import pandas as pd
import numpy as np

# Datos simulados
data = {
    "Proyecto": ["ERP-Alpha", "ERP-Alpha", "CRM-Beta", "CRM-Beta", "CRM-Beta",
                 "Ecom-Gamma", "Ecom-Gamma", "Ecom-Gamma", "Ecom-Gamma", "ERP-Alpha"],
    "Empleado": ["Luis", "Ana", "Carlos", "Luis", "Sofía", "Ana", "Sofía", "Carlos", "Luis", "Carlos"],
    "Departamento": ["Backend", "Backend", "Frontend", "Backend", "Frontend",
                     "QA", "QA", "Backend", "Backend", "Frontend"],
    "Horas": [42, 38, 45, 40, 44, 36, 38, 43, 41, 39],
    "Costo por Hora": [30, 35, 28, 30, 28, 25, 25, 30, 30, 28],
    "Sprint": [1, 1, 1, 2, 2, 1, 2, 2, 3, 3]
}

df = pd.DataFrame(data)#convierte la data en un dataframe llamado df
df["Costo Total"] = df["Horas"] * df["Costo por Hora"]#genera una nueva columna que implica la multiplicacion de horas x costo por hora
df#imprime el dataframe


,Proyecto,Empleado,Departamento,Horas,Costo por Hora,Sprint,Costo Total
0,ERP-Alpha,Luis,Backend,42,30,1,1260
1,ERP-Alpha,Ana,Backend,38,35,1,1330
2,CRM-Beta,Carlos,Frontend,45,28,1,1260
3,CRM-Beta,Luis,Backend,40,30,2,1200
4,CRM-Beta,Sofía,Frontend,44,28,2,1232
5,Ecom-Gamma,Ana,QA,36,25,1,900
6,Ecom-Gamma,Sofía,QA,38,25,2,950
7,Ecom-Gamma,Carlos,Backend,43,30,2,1290
8,Ecom-Gamma,Luis,Backend,41,30,3,1230
9,ERP-Alpha,Carlos,Frontend,39,28,3,1092


#PARTE1



Agrupa los datos por Proyecto y Sprint.

Calcula para cada grupo:

Horas totales

Costo total

Costo promedio por hora

Número de empleados únicos

Crea una tabla con estos datos.




In [ ]:
resultado=df.groupby(["Proyecto","Sprint"]).agg(
    Horas_Totales=("Horas","sum"),
    Costo_Total=("Costo Total","sum"),
    Empleados_Unicos=("Empleado",pd.Series.nunique)


)
resultado["Costo_Promedio_Hora"]=resultado["Costo_Total"]/resultado["Horas_Totales"]
resultado

Horas_Totales  Costo_Total  Empleados_Unicos  \
Proyecto   Sprint                                                 
CRM-Beta   1                  45         1260                 1   
           2                  84         2432                 2   
ERP-Alpha  1                  80         2590                 2   
           3                  39         1092                 1   
Ecom-Gamma 1                  36          900                 1   
           2                  81         2240                 2   
           3                  41         1230                 1   

                   Costo_Promedio_Hora  
Proyecto   Sprint                       
CRM-Beta   1                 28.000000  
           2                 28.952381  
ERP-Alpha  1                 32.375000  
           3                 28.000000  
Ecom-Gamma 1                 25.000000  
           2                 27.654321  
           3                 30.000000

Filtra los proyectos que en algún sprint:

Hayan superado los $1300 de costo total, y

Tengan un costo promedio por hora superior a $32.

In [ ]:
sospechoso=resultado[(resultado["Costo_Total"]>1300) & (resultado["Costo_Promedio_Hora"]>32)]
sospechoso

,,Horas_Totales,Costo_Total,Empleados_Unicos,Costo_Promedio_Hora
Proyecto,Sprint,,,,
ERP-Alpha,1,80,2590,2,32.375


In [ ]:
resultado_sprint1 = resultado.xs(1, level="Sprint")  # Selecciona solo el Sprint 1
resultado_sprint1 = resultado_sprint1.sort_values(by="Horas_Totales", ascending=False)

print(resultado_sprint1)


            Horas_Totales  Costo_Total  Empleados_Unicos  Costo_Promedio_Hora
Proyecto                                                                     
ERP-Alpha              80         2590                 2               32.375
CRM-Beta               45         1260                 1               28.000
Ecom-Gamma             36          900                 1               25.000


In [ ]:
df.sort_values(by="Costo por Hora", ascending=False).head(3)


,Proyecto,Empleado,Departamento,Horas,Costo por Hora,Sprint,Costo Total
1,ERP-Alpha,Ana,Backend,38,35,1,1330
0,ERP-Alpha,Luis,Backend,42,30,1,1260
3,CRM-Beta,Luis,Backend,40,30,2,1200


Comenta los posibles motivos de estos sobrecostos.


El sobrecosto del proyecto se puede deber a que se trabajó con 2 empleados en un solo sprint, con una cantidad de horas totales alta respecto a los demás que realizaron un sprint, además de que el costo por hora de ambos son los mas altos

Exporta la tabla final a un CSV con el nombre: proyectos_sospechosos.csv.

In [ ]:
sospechoso.to_csv("proyectos_sospechosos.csv")

#Sistema de Gestión de Proyectos (Clase Empleados)

In [ ]:
class EmpleadoProyecto:
    def __init__(self, nombre, departamento):
        self.nombre = nombre
        self.departamento = departamento
        self.participaciones = []

    def agregar_participacion(self, proyecto, horas, costo, sprint):
        self.participaciones.append({
            "proyecto": proyecto,
            "horas": horas,
            "costo_total": horas * costo,
            "sprint": sprint
        })

    def total_horas(self):
        return sum(p["horas"] for p in self.participaciones)

    def costo_total(self):
        return sum(p["costo_total"] for p in self.participaciones)

    def sprints(self):
        return len(set(p["sprint"] for p in self.participaciones))

    def resumen(self):
      proyectos_unicos = len(set(p["proyecto"] for p in self.participaciones))
      return {
        "Empleado": self.nombre,
        "Departamento": self.departamento,
        "Costo Total": self.costo_total(),
        "Horas Totales": self.total_horas(),
        "Sprints Participados": self.sprints(),
        "Proyectos Distintos": proyectos_unicos,
        "Promedio Costo/Hora": self.promedio_horas(),
        "eficiencia": self.eficiencia()
    }

    def promedio_horas(self):
        total_horas = self.total_horas()
        costo_total = self.costo_total()
        if total_horas == 0:
            return 0
        else:
            return (costo_total / total_horas)

     #Agrega un método llamado eficiencia() en la clase EmpleadoProyecto, que calcule: Eficiencia=(Total de Horas/Costo Total)×100
    def eficiencia(self):
        total_horas = self.total_horas()
        costo_total = self.costo_total()
        if total_horas == 0:
            return 0
        return (total_horas / costo_total) * 100


#PARTE2







**Objetivo**: Evaluar el rendimiento de cada empleado con clases orientadas a objetos, considerando métricas financieras y de carga laboral.









Usa la clase EmpleadoProyecto y el sistema SistemaProyectos.

Para cada empleado, calcula:

Costo total invertido

Total de horas trabajadas

Número de proyectos distintos

Promedio de costo/hora

In [ ]:
sistema = SistemaProyectos(df)  # df es tu DataFrame original
resumen = sistema.resumen_empleados()
resumen

,Empleado,Departamento,Costo Total,Horas Totales,Sprints Participados,Proyectos Distintos,Promedio Costo/Hora,eficiencia
0,Luis,Backend,3690,123,3,3,30.000000,3.333333
2,Carlos,Frontend,3642,127,3,3,28.677165,3.487095
1,Ana,Backend,2230,74,1,2,30.135135,3.318386
3,Sofía,Frontend,2182,82,1,2,26.609756,3.758020


Crea un ranking de empleados según su eficiencia.

Exporta los 3 empleados más eficientes a un archivo llamado top_eficientes.csv.

In [ ]:
top3=resumen.sort_values("eficiencia", ascending=False).head(3)
top3.to_csv("top_eficientes.csv")
top3

,Empleado,Departamento,Costo Total,Horas Totales,Sprints Participados,Proyectos Distintos,Promedio Costo/Hora,eficiencia
3,Sofía,Frontend,2182,82,1,2,26.609756,3.758020
2,Carlos,Frontend,3642,127,3,3,28.677165,3.487095
0,Luis,Backend,3690,123,3,3,30.000000,3.333333


#Sistema de Gestión de Proyectos (Clase Sistema Proyectos)

In [ ]:
class SistemaProyectos:
    def __init__(self, df):
        self.df = df
        self.empleados = self._cargar_empleados()

    def _cargar_empleados(self):
        empleados = {}
        for _, row in self.df.iterrows():
            nombre = row["Empleado"]
            if nombre not in empleados:
                empleados[nombre] = EmpleadoProyecto(nombre, row["Departamento"])
            empleados[nombre].agregar_participacion(row["Proyecto"], row["Horas"],
                                                     row["Costo por Hora"], row["Sprint"])
        return empleados

    def resumen_empleados(self):
        data = [emp.resumen() for emp in self.empleados.values()]
        return pd.DataFrame(data).sort_values("Costo Total", ascending=False)

    def horas_por_proyecto(self):
        return self.df.groupby("Proyecto")["Horas"].sum()

    def empleados_por_sprint(self):
        return self.df.groupby("Sprint")["Empleado"].nunique()

    def carga_media_por_empleado(self):
        return self.df.groupby("Empleado")["Horas"].mean()

    def mostrar_resumen(self):
        print("📊 Horas por Proyecto:\n", self.horas_por_proyecto(), "\n")
        print("👥 Empleados por Sprint:\n", self.empleados_por_sprint(), "\n")
        return self.resumen_empleados()


    def alerta_sobrecarga_empleados(self):#segun instrucciones en Parte3, creacion de la funcion para notificar sobre carga en empleados
      resumen = self.resumen_empleados()
      promedio_horas = resumen["Horas Totales"].mean()
      df_alerta = resumen[["Empleado", "Horas Totales", "Sprints Participados"]].copy()
      df_alerta["% Carga respecto a la media"] = (df_alerta["Horas Totales"] / promedio_horas) * 100
      sobrecargados = df_alerta[
          (df_alerta["Sprints Participados"] > 2) &
          (df_alerta["% Carga respecto a la media"] > 120)
      ]

      return sobrecargados

#PARTE3

**Objetivo**: Identificar desequilibrios en la distribución de tareas entre empleados.








Calcula la carga media de trabajo (en horas) por empleado.

In [ ]:
sistema = SistemaProyectos(df)  # df es tu DataFrame original
resumen = sistema.carga_media_por_empleado()
resumen

,Horas
Empleado,
Ana,37.000000
Carlos,42.333333
Luis,41.000000
Sofía,41.000000


Detecta a los empleados que:

Participaron en más de 3 sprints



In [ ]:
sistema = SistemaProyectos(df)  # df es tu DataFrame original
resumen = sistema.resumen_empleados()
resumen[(resumen["Sprints Participados"]>3)]

,Empleado,Departamento,Costo Total,Horas Totales,Sprints Participados,Proyectos Distintos,Promedio Costo/Hora,eficiencia


Tienen una carga mayor al 20% sobre el promedio general

In [ ]:
sistema = SistemaProyectos(df)
resumen = sistema.resumen_empleados()
resumen
promedio_horas = resumen["Horas Totales"].mean()#Calcular el promedio de horas totales
umbral = promedio_horas * 1.2# Filtrar los empleados que superan ese promedio en más de un 20%
empleados_sobrecargados = resumen[resumen["Horas Totales"] > umbral]
print(f"Promedio de horas totales: {promedio_horas:.2f}")
empleados_sobrecargados

Promedio de horas totales: 101.50


,Empleado,Departamento,Costo Total,Horas Totales,Sprints Participados,Proyectos Distintos,Promedio Costo/Hora,eficiencia
0,Luis,Backend,3690,123,3,3,30.000000,3.333333
2,Carlos,Frontend,3642,127,3,3,28.677165,3.487095


Crea un DataFrame con:

Nombre

Horas totales

Número de sprints

Porcentaje de carga respecto a la media


In [ ]:
resumen = sistema.resumen_empleados()# Obtener resumen general
promedio_horas = resumen["Horas Totales"].mean()# Calcular promedio general de horas
df_alerta = resumen[["Empleado", "Horas Totales", "Sprints Participados"]].copy()# Crear un nuevo DataFrame con las columnas solicitadas
df_alerta["% Carga respecto a la media"] = (df_alerta["Horas Totales"] / promedio_horas) * 100# Agregar columna con el porcentaje de carga respecto al promedio
df_alerta


,Empleado,Horas Totales,Sprints Participados,% Carga respecto a la media
0,Luis,123,3,121.182266
2,Carlos,127,3,125.123153
1,Ana,74,1,72.906404
3,Sofía,82,1,80.788177


Añade un método a la clase SistemaProyectos para automatizar esta alerta.


In [ ]:
sistema = SistemaProyectos(df)
sistema.alerta_sobrecarga_empleados()


,Empleado,Horas Totales,Sprints Participados,% Carga respecto a la media
0,Luis,123,3,121.182266
2,Carlos,127,3,125.123153


#PARTE4

**Objetivo**: Modelar el impacto de una mejora en procesos (por ejemplo, reducción del 10% en costo por hora).












Crea una copia del DataFrame original.

In [ ]:
df_mejorado = df.copy()
print("DataFrame original copiado correctamente")

DataFrame original copiado correctamente


Aplica un descuento del 10% al costo por hora de todos los registros.

In [ ]:
df_mejorado["Costo por Hora"] = df_mejorado["Costo por Hora"] * 0.9
print("Descuento del 10% aplicado al costo por hora")
print(df_mejorado[["Empleado", "Costo por Hora"]])

Descuento del 10% aplicado al costo por hora
  Empleado  Costo por Hora
0     Luis            27.0
1      Ana            31.5
2   Carlos            25.2
3     Luis            27.0
4    Sofía            25.2
5      Ana            22.5
6    Sofía            22.5
7   Carlos            27.0
8     Luis            27.0
9   Carlos            25.2


Recalcula el costo total.

In [ ]:
df_mejorado["Costo Total"] = df_mejorado["Horas"] * df_mejorado["Costo por Hora"]
print("Costo total recalculado")
print(df_mejorado[["Empleado", "Horas", "Costo por Hora", "Costo Total"]])

Costo total recalculado
  Empleado  Horas  Costo por Hora  Costo Total
0     Luis     42            27.0       1134.0
1      Ana     38            31.5       1197.0
2   Carlos     45            25.2       1134.0
3     Luis     40            27.0       1080.0
4    Sofía     44            25.2       1108.8
5      Ana     36            22.5        810.0
6    Sofía     38            22.5        855.0
7   Carlos     43            27.0       1161.0
8     Luis     41            27.0       1107.0
9   Carlos     39            25.2        982.8


Vuelve a realizar el análisis de eficiencia por sprint.

In [ ]:
# Función para analizar eficiencia por sprint
def analizar_eficiencia_sprint(dataframe, nombre="DataFrame"):
    # Calcular horas y costos totales por sprint
    sprint_stats = dataframe.groupby("Sprint").agg({
        "Horas": "sum",
        "Costo Total": "sum"
    }).reset_index()

    # Calcular costo promedio por hora para cada sprint
    sprint_stats["Costo Promedio por Hora"] = sprint_stats["Costo Total"] / sprint_stats["Horas"]

    # Determinar eficiencia: Un sprint es ineficiente si su costo promedio por hora es mayor al promedio general
    costo_promedio_general = dataframe["Costo Total"].sum() / dataframe["Horas"].sum()
    sprint_stats["Eficiente"] = sprint_stats["Costo Promedio por Hora"] <= costo_promedio_general

    print(f"\nAnálisis de eficiencia para {nombre}:")
    print(sprint_stats)
    print(f"Costo promedio general por hora: {costo_promedio_general:.2f}")
    sprints_ineficientes = sum(~sprint_stats["Eficiente"])
    print(f"Número de sprints ineficientes: {sprints_ineficientes}")

    return sprint_stats, costo_promedio_general, sprints_ineficientes

# Analizar DataFrame original
stats_original, costo_prom_original, ineficientes_original = analizar_eficiencia_sprint(df, "DataFrame Original")

# Analizar DataFrame mejorado
stats_mejorado, costo_prom_mejorado, ineficientes_mejorado = analizar_eficiencia_sprint(df_mejorado, "DataFrame Mejorado")


Análisis de eficiencia para DataFrame Original:
   Sprint  Horas  Costo Total  Costo Promedio por Hora  Eficiente
0       1    161         4750                29.503106      False
1       2    165         4672                28.315152       True
2       3     80         2322                29.025000      False
Costo promedio general por hora: 28.93
Número de sprints ineficientes: 2

Análisis de eficiencia para DataFrame Mejorado:
   Sprint  Horas  Costo Total  Costo Promedio por Hora  Eficiente
0       1    161       4275.0                26.552795      False
1       2    165       4204.8                25.483636       True
2       3     80       2089.8                26.122500      False
Costo promedio general por hora: 26.03
Número de sprints ineficientes: 2


Compara los resultados antes y después:

¿Cuántos sprints dejaron de ser ineficientes?

¿En qué medida disminuyó el gasto?

In [ ]:
# Comparar resultados
print("\n--- Comparación de Resultados ---")

# sprints ineficientes
reduccion_sprints_ineficientes = ineficientes_original - ineficientes_mejorado
print(f"Sprints que dejaron de ser ineficientes: {reduccion_sprints_ineficientes}")

# disminución de gastos
reduccion_gasto = df["Costo Total"].sum() - df_mejorado["Costo Total"].sum()
porcentaje_reduccion = (reduccion_gasto / df["Costo Total"].sum()) * 100

print(f"Reducción del gasto total: ${reduccion_gasto:.2f}")
print(f"Porcentaje de reducción del gasto: {porcentaje_reduccion:.2f}%")

# Compraración detallada
comparacion = pd.merge(stats_original, stats_mejorado, on="Sprint", suffixes=("_original", "_mejorado"))
print("\nComparación detallada por sprint:")
print(comparacion[["Sprint", "Costo Total_original", "Costo Total_mejorado", "Eficiente_original", "Eficiente_mejorado"]])


--- Comparación de Resultados ---
Sprints que dejaron de ser ineficientes: 0
Reducción del gasto total: $1174.40
Porcentaje de reducción del gasto: 10.00%

Comparación detallada por sprint:
   Sprint  Costo Total_original  Costo Total_mejorado  Eficiente_original  \
0       1                  4750                4275.0               False   
1       2                  4672                4204.8                True   
2       3                  2322                2089.8               False   

   Eficiente_mejorado  
0               False  
1                True  
2               False  
